In [1]:
import pandas as pd

In [4]:
train = pd.read_csv(r'train_data.csv',index_col = 0)
test = pd.read_csv(r'test_data.csv',index_col = 0)

In [5]:
train.head(10)

,url,title,label
ID,,,
0,m.kp.md,"Экс-министр экономики Молдовы - главе МИДЭИ, ц...",0
1,www.kp.by,Эта песня стала известна многим телезрителям б...,0
2,fanserials.tv,Банши 4 сезон 2 серия Бремя красоты смотреть о...,0
3,colorbox.spb.ru,Не Беси Меня Картинки,0
4,tula-sport.ru,В Новомосковске сыграют следж-хоккеисты алекси...,0
5,beregifiguru.ru,Салат: корейская морковь + копченая курица + к...,0
6,ekb.vseinstrumenti.ru,"Угловой пневмогайковерт FUBAG RWC 105 1/2"" 100...",0
7,mirtabaka.org,Табачный магазин `Мир Табака` | КУРИТЕЛЬНЫЕ ТР...,0
8,xlecx.com,league of legends » Page 5 » Porn comics free ...,1
